In [113]:
import re
import networkx as nx 

def parse_instruction(raw_instruction):
    regex = r"Step (\S) must be finished before step (\S) can begin."
    node_from, node_to = re.findall(regex, raw_instruction)[0]
    
    return node_from, node_to

def build_graph(raw_instruction_list):
    graph = nx.DiGraph()
    
    for raw_instruction in raw_instruction_list:
        node_from, node_to = parse_instruction(raw_instruction)
        graph.add_edge(node_from, node_to)
    
    return graph

def alphabetical_topological_sort(graph):
    indegree_map = dict()
    zero_indegree = list()
    all_indegree_map = graph.in_degree()
    for node in all_indegree_map:
        if all_indegree_map[node] > 0:
            indegree_map[node] = all_indegree_map[node]
        else:
            zero_indegree.append(node)

    while zero_indegree:
        zero_indegree=sorted(zero_indegree, reverse=True)
        node = zero_indegree.pop()

        for _, child in graph.edges(node):
            indegree_map[child] -= 1
            
            if indegree_map[child] == 0:
                zero_indegree.append(child)
                del indegree_map[child]
        
        yield node

def part_1_solution(input_list):
    graph = build_graph(input_list)
    solution = "".join(alphabetical_topological_sort(graph))

    return solution

In [115]:
test_input = """Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin."""

assert(part_1_solution(test_input.splitlines()) == 'CABDFE')
print("Test passed")

Test passed


In [116]:
with open("inputs/Day_07.txt") as f:
    puzzle_input = f.readlines()

In [117]:
print(f"Part 1 solution: {part_1_solution(puzzle_input)}")

Part 1 solution: FDSEGJLPKNRYOAMQIUHTCVWZXB


In [118]:
def simulate_graph_execution(graph, number_of_workers):
    avaliable_nodes = list()
    current_tasks = list()
    done_nodes = list()
    elapsed_ticks = 0
    indegree_map = graph.in_degree()
    not_processed_nodes = set(graph.nodes())

    for node in indegree_map:
        if indegree_map[node] == 0:
            avaliable_nodes.append(node)

    while not_processed_nodes:
        # Sort and then pop to get nodes in alphabetical order (when tie)
        avaliable_nodes = sorted(avaliable_nodes, reverse=True)

        # Get new tasks in possible
        while len(current_tasks) < number_of_workers:
            if not avaliable_nodes:
                break

            node = avaliable_nodes.pop()
            new_task = (node, ord(node) - 64 + 60)
            current_tasks.append(new_task)

        # Process current tasks until some is done
        while not done_nodes:
            # Update times for all current tasks
            current_tasks = list(map(lambda task: (task[0], task[1] - 1), current_tasks))
            # Check if some of the tasks are done
            done_nodes = [node for node, remaining_time in current_tasks if remaining_time == 0]
            # Remove done tasks from current tasks
            current_tasks = [(node, remaining_time) for node, remaining_time in current_tasks
                             if remaining_time != 0]
            elapsed_ticks += 1

        # Search for avaliable nodes
        while done_nodes:
            node = done_nodes.pop()
            not_processed_nodes.remove(node)

            for _, child in graph.edges(node):
                indegree_map[child] -= 1

                if indegree_map[child] == 0:
                    avaliable_nodes.append(child)

    return elapsed_ticks


def part_2_solution(input_list, number_of_workers):
    graph = build_graph(input_list)

    return simulate_graph_execution(graph, number_of_workers)

In [119]:
print(f"Part 2 solution: {part_2_solution(puzzle_input, 5)}")

Part 2 solution: 1000


In [120]:
%%timeit
part_1_solution(puzzle_input)

864 µs ± 49.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [122]:
%%timeit
part_2_solution(puzzle_input, 5)

7.02 ms ± 883 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
